In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import utils as data_analysis

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from ml_test import Disease_Prediction

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [ ]:
df = pd.read_csv('cardio_train.csv', sep=';', index_col=0)

In [ ]:
data_analysis.bmi(df)

In [ ]:
# outliers bmi
Q1, Q3 = np.quantile(df['bmi'], 0.25), np.quantile(df['bmi'], 0.75)
IQR = Q3 - Q1
threshold = 3
df = df[(df['bmi'] >= Q1 - threshold * IQR) & ( df['bmi'] <= Q3 + threshold * IQR)]

In [ ]:
data_analysis.bmi_cat(df)

In [ ]:
### ?????????????????

df = df[df['bmi'] >= 12]

In [ ]:
# outliers ap_hi
Q1, Q3 = np.quantile(df['ap_hi'], 0.25), np.quantile(df['ap_hi'], 0.75)
IQR = Q3 - Q1
threshold = 1.5
df = df[(df['ap_hi'] >= Q1 - threshold * IQR) & ( df['ap_hi'] <= Q3 + threshold * IQR)]

In [ ]:
# outliers ap_lo
Q1, Q3 = np.quantile(df['ap_lo'], 0.25), np.quantile(df['ap_lo'], 0.75)
IQR = Q3 - Q1
threshold = 1.5
df = df[(df['ap_lo'] >= Q1 - threshold * IQR) & ( df['ap_lo'] <= Q3 + threshold * IQR)]

In [ ]:
df = df.dropna()

In [ ]:
data_analysis.pressure_cat(df)

In [ ]:
df_new = df.copy()

In [ ]:
df_new_01 = df_new.drop(columns=['ap_hi', 'ap_lo', 'height', 'weight', 'bmi'])
df_new_01 = pd.get_dummies(df_new_01, columns = ["bmi_cat", 'pressure_cat',  "gender"], drop_first=True)

In [ ]:
df_new_02 = df_new.drop(columns=['bmi_cat', 'pressure_cat', 'height', 'weight'])
df_new_02 = pd.get_dummies(df_new_02, columns = ["gender"], drop_first=True)
df_new_02.head()

## DF_NEW_01
---

#### Logistic regression.
---

In [ ]:
model = Disease_Prediction(df_new_01, 'cardio')
model.voting_clf()

In [ ]:
model.log_reg()

### KNN
---

In [ ]:
model.knn()

### Random Forest.
---

In [ ]:
model.random_forest()

## DF_NEW_02
---

### Logistic regression.
---

In [ ]:
#model = Disease_Prediction(df_new_02, 'cardio')
#model.log_reg()

### KNN.
---

In [ ]:
#model.knn()

### Random Forest.
---

In [ ]:
#model.random_forest()